# Librerías

Ejecutar esta celda siempre al comienzo. Si da error, reiniciar el entorno de ejecución.

In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

# Clone datasets
!apt-get install git
!git clone --depth=1 --filter=blob:none --sparse https://github.com/raulcastillabravo/datasets.git
%cd datasets
!git sparse-checkout set renfe/passengers/ renfe/stations/
%cd ..

import os
import sys
from datetime import datetime
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Our First Spark Example") \
       .getOrCreate()

spark

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,032 kB]
G

# Esquemas

In [2]:
PASSENGERS_SCHEMA =  T.StructType([
    T.StructField('codigo_estacion', T.IntegerType(), True),
    T.StructField('nombre_estacion', T.StringType(), True),
    T.StructField('nucleo_cercanias', T.StringType(), True),
    T.StructField('tramo_horario', T.StringType(), True),
    T.StructField('viajeros_subidos', T.IntegerType(), True),
    T.StructField('viajeros_bajados', T.IntegerType(), True),
])

STATIONS_SCHEMA = T.StructType([
    T.StructField("codigo_estacion", T.IntegerType(), True),
    T.StructField("descripcion", T.StringType(), True),
    T.StructField("latitud", T.DoubleType(), True),
    T.StructField("longitud", T.DoubleType(), True),
    T.StructField("direccion", T.StringType(), True),
    T.StructField("cp", T.StringType(), True),
    T.StructField("poblacion", T.StringType(), True),
    T.StructField("provincia", T.StringType(), True),
    T.StructField("fichas", T.StringType(), True),
    T.StructField("tuneles_lavado", T.StringType(), True),
])

# Lectura

In [3]:
df_passengers = spark.read.csv('/content/datasets/renfe/passengers/', sep=';', header=True, schema=PASSENGERS_SCHEMA)
df_stations = spark.read.csv('/content/datasets/renfe/stations/', sep=';', header=True, schema=STATIONS_SCHEMA)

Con **createOrReplaceTempView** creamos una vista temporal a partir de nuestro DataFrame, para que a partir de ese momento podamos lanzar consultas SQL sobre nuestros datos

In [4]:
df_passengers.createOrReplaceTempView("passengers")
df_stations.createOrReplaceTempView("stations")

In [5]:
spark.sql("SELECT * FROM passengers").show(truncate=False)

+---------------+----------------------+----------------+-------------+----------------+----------------+
|codigo_estacion|nombre_estacion       |nucleo_cercanias|tramo_horario|viajeros_subidos|viajeros_bajados|
+---------------+----------------------+----------------+-------------+----------------+----------------+
|71600          |SANT VICENÇ DE CALDERS|BARCELONA       |00:00 - 00:30|0               |19              |
|71600          |SANT VICENÇ DE CALDERS|BARCELONA       |00:30 - 01:00|0               |1               |
|71600          |SANT VICENÇ DE CALDERS|BARCELONA       |05:00 - 05:30|43              |0               |
|71600          |SANT VICENÇ DE CALDERS|BARCELONA       |05:30 - 06:00|46              |0               |
|71600          |SANT VICENÇ DE CALDERS|BARCELONA       |06:00 - 06:30|56              |0               |
|71600          |SANT VICENÇ DE CALDERS|BARCELONA       |06:30 - 07:00|187             |22              |
|71600          |SANT VICENÇ DE CALDERS|BARCEL

In [6]:
spark.sql("SELECT * FROM stations").show(truncate=False)

+---------------+------------------------------+----------+---------+----------------------------------------+-----+---------------------------+---------+--------------------------------------------------------------+--------------+
|codigo_estacion|descripcion                   |latitud   |longitud |direccion                               |cp   |poblacion                  |provincia|fichas                                                        |tuneles_lavado|
+---------------+------------------------------+----------+---------+----------------------------------------+-----+---------------------------+---------+--------------------------------------------------------------+--------------+
|72400          |AEROPORT                      |41.3038736|2.0724734|ZONA AEROPUERTO, S/N                    |8820 |Prat de Llobregat, El      |Barcelona|https://data.renfe.com/files/estaciones/FichaEstacion72400.pdf|NULL          |
|78804          |ARC DE TRIOMF                 |41.3924353|2.1804163

# Ejercicio 1

Escribe la tabla **passengers** particionada por la primera hora del tramo horario. Si **tramo_horario** es 09:00 - 09:30, la partición debe ser **tramo_inicio=0900**

**Funciones útiles**

```
F.split(F.col('colname'), ',')[0]  # divide los valores por coma y extrae el primero
F.regexp_replace(F.col('colname'), ',', ';')  # Cambia las comas por puntos y comas
```



In [7]:
df = spark.sql("""
  SELECT
    *,
    regexp_replace(split(tramo_horario, ' - ')[0], ':', '') AS tramo_inicio
  FROM passengers
""")

df.show(truncate=False)

+---------------+----------------------+----------------+-------------+----------------+----------------+------------+
|codigo_estacion|nombre_estacion       |nucleo_cercanias|tramo_horario|viajeros_subidos|viajeros_bajados|tramo_inicio|
+---------------+----------------------+----------------+-------------+----------------+----------------+------------+
|71600          |SANT VICENÇ DE CALDERS|BARCELONA       |00:00 - 00:30|0               |19              |0000        |
|71600          |SANT VICENÇ DE CALDERS|BARCELONA       |00:30 - 01:00|0               |1               |0030        |
|71600          |SANT VICENÇ DE CALDERS|BARCELONA       |05:00 - 05:30|43              |0               |0500        |
|71600          |SANT VICENÇ DE CALDERS|BARCELONA       |05:30 - 06:00|46              |0               |0530        |
|71600          |SANT VICENÇ DE CALDERS|BARCELONA       |06:00 - 06:30|56              |0               |0600        |
|71600          |SANT VICENÇ DE CALDERS|BARCELON

In [8]:
df.write.partitionBy('tramo_inicio').parquet('/content/exercise1/')

# Ejercicio 2

Calcula el total de viajeros que han subido y bajado por núcleo de cercanías

**Funciones útiles**


```
F.sum(F.col('colname)).alias('new_colname') # Permite sumar los valores de una columna
```



In [9]:
df = spark.sql("""
  SELECT
    nucleo_cercanias,
    SUM(viajeros_subidos) AS total_viajeros_subidos,
    SUM(viajeros_bajados) AS total_viajeros_bajados
  FROM passengers
  GROUP BY nucleo_cercanias
""")

df.show(truncate=False)

+----------------+----------------------+----------------------+
|nucleo_cercanias|total_viajeros_subidos|total_viajeros_bajados|
+----------------+----------------------+----------------------+
|SAN SEBASTIÁN   |17548                 |17548                 |
|MADRID          |907489                |907489                |
|BARCELONA       |369477                |369477                |
|CANTABRIA AM    |10801                 |10801                 |
|ASTURIAS        |17623                 |17623                 |
|SEVILLA         |28424                 |28424                 |
|BILBAO          |33394                 |33394                 |
|VALENCIA        |46873                 |46873                 |
|ASTURIAS AM     |7408                  |7408                  |
|NULL            |NULL                  |NULL                  |
|MÁLAGA          |27404                 |27404                 |
|PAIS VASCO AM   |2906                  |2906                  |
|CÁDIZ           |7756   

# Ejercicio 3

Filtra los tramos horarios que tengan como **hora de inicio una hora en punto**. Si el tramo horario es 00:00 - 00:30, sí lo queremos, pero si es 00:30 - 01:00, no.



In [10]:
df = spark.sql("""
  SELECT *
  FROM (
    SELECT
      *,
      split(tramo_horario, ' - ')[0] AS tramo_inicio,
      split(split(tramo_horario, ' - ')[0], ':')[1] AS minutos_tramo_inicio
    FROM passengers
  )
  WHERE minutos_tramo_inicio = '00'
""")

df.show(truncate=False)

+---------------+----------------------+----------------+-------------+----------------+----------------+------------+--------------------+
|codigo_estacion|nombre_estacion       |nucleo_cercanias|tramo_horario|viajeros_subidos|viajeros_bajados|tramo_inicio|minutos_tramo_inicio|
+---------------+----------------------+----------------+-------------+----------------+----------------+------------+--------------------+
|71600          |SANT VICENÇ DE CALDERS|BARCELONA       |00:00 - 00:30|0               |19              |00:00       |00                  |
|71600          |SANT VICENÇ DE CALDERS|BARCELONA       |05:00 - 05:30|43              |0               |05:00       |00                  |
|71600          |SANT VICENÇ DE CALDERS|BARCELONA       |06:00 - 06:30|56              |0               |06:00       |00                  |
|71600          |SANT VICENÇ DE CALDERS|BARCELONA       |07:00 - 07:30|45              |50              |07:00       |00                  |
|71600          |SAN

# Ejercicio 4

Calcula cuántos viajeros se suben a los trenes en cada población y ordena el resultado de mayor a menor.

**Pista**: tendrás que cruzar (**join**) la tabla de pasajeros con la de estaciones, agrupar y sumar. Puedes ordenar el resultado final usando **orderBy** de la siguiente forma


```
df.orderBy(F.col('colname'), ascending=False)  # Ordena de forma descendente
```



In [11]:
df = spark.sql("""
  SELECT *
  FROM passengers p
  LEFT JOIN stations s ON p.codigo_estacion = s.codigo_estacion
""")

df.show()

+---------------+--------------------+----------------+-------------+----------------+----------------+---------------+--------------------+----------+---------+--------------------+-----+------------+---------+--------------------+--------------+
|codigo_estacion|     nombre_estacion|nucleo_cercanias|tramo_horario|viajeros_subidos|viajeros_bajados|codigo_estacion|         descripcion|   latitud| longitud|           direccion|   cp|   poblacion|provincia|              fichas|tuneles_lavado|
+---------------+--------------------+----------------+-------------+----------------+----------------+---------------+--------------------+----------+---------+--------------------+-----+------------+---------+--------------------+--------------+
|          71600|SANT VICENÇ DE CA...|       BARCELONA|00:00 - 00:30|               0|              19|          71600|SANT VICENÇ DE CA...|41.1861054|1.5245117|CALLE ESTACIO FER...|43880|Vendrell, El|Tarragona|https://data.renf...|          NULL|
|       

In [12]:
df = spark.sql("""
  SELECT
    poblacion,
    SUM(viajeros_subidos) AS total_viajeros_subidos
  FROM passengers p
  LEFT JOIN stations s ON p.codigo_estacion = s.codigo_estacion
  GROUP BY poblacion
  ORDER BY total_viajeros_subidos DESC
""")

df.show()

+--------------------+----------------------+
|           poblacion|total_viajeros_subidos|
+--------------------+----------------------+
|              Madrid|                602039|
|           Barcelona|                145742|
|              Getafe|                 36170|
|             Leganés|                 28553|
|         Fuenlabrada|                 28045|
|   Alcalá de Henares|                 22980|
|               Parla|                 20748|
|            Móstoles|                 19263|
|            Valencia|                 18309|
|            Alcorcón|                 17615|
|              Bilbao|                 17583|
|                NULL|                 17098|
|   Torrejón de Ardoz|                 15336|
|             Sevilla|                 15262|
|            Sabadell|                 13620|
|             Coslada|                 11765|
|Rozas de Madrid, Las|                 11587|
|              Málaga|                 11549|
|Hospitalet de Llo...|            

# Ejercicio 5 (micro proyecto)

Se ha detectado que en la tabla de estaciones hay fichas que están vacías (**NULL**). Se quiere evaluar el impacto de estas fichas vacías para saber a cuántos pasajeros les está afectando.

Para ello, se pide realizar un proceso automático que genere un informe con las siguientes características:

1. Debe mostrar la información agrupada por núcleo de cercanías.
2. Debe calcular dos KPIs:
  1. El porcentaje de viajeros que han subido a un tren sin ficha.
  2. El número de viajeros que han subido a un tren sin ficha.
3. El resultado debe estar ordenado de mayor a menor porcentaje.
4. El resultado debe escribirse particionado por núcleo de cercanías.
5. Cada vez que se ejecute el proceso, debe crearse una partición nueva con los resultados de la ejecución para poder tener trazabilidad de análisis anteriores.

**Pistas**

```
df.filter(F.col('colname').isNull())  # Filtra los valores NULL
df.filter(F.col('colname').isNotNull())  # Filtra los valores no NULL


# Permite crear una marca de tiempo con el instante de ejecucion
from datetime import datetime
now = datetime.now().strftime("%Y%m%d%H%M%S") # El resultado es un string
```



## Fase 1.

1. Cruzar tabla **df_passengers** con **df_station**.
2. Filtrar los casos donde las fichas son nulas y donde no lo son para crear dos DataFrames llamados **df_null** y **df_not_null**.

In [13]:
df = spark.sql("""
  SELECT *
  FROM passengers p
  LEFT JOIN stations s ON p.codigo_estacion = s.codigo_estacion
""")

df.createOrReplaceTempView("joined_data")

df_null = spark.sql("SELECT * FROM joined_data WHERE fichas IS NULL")
df_not_null = spark.sql("SELECT * FROM joined_data WHERE fichas IS NOT NULL")

df_null.show()

+---------------+---------------+----------------+-------------+----------------+----------------+---------------+---------------+---------+---------+--------------------+----+---------+---------+------+--------------+
|codigo_estacion|nombre_estacion|nucleo_cercanias|tramo_horario|viajeros_subidos|viajeros_bajados|codigo_estacion|    descripcion|  latitud| longitud|           direccion|  cp|poblacion|provincia|fichas|tuneles_lavado|
+---------------+---------------+----------------+-------------+----------------+----------------+---------------+---------------+---------+---------+--------------------+----+---------+---------+------+--------------+
|          71801|BARCELONA-SANTS|       BARCELONA|00:00 - 00:30|              39|              10|          71801|BARCELONA-SANTS|41.379335|2.1394773|PLAÇA DELS PAISOS...|8014|Barcelona|Barcelona|  NULL|          NULL|
|          71801|BARCELONA-SANTS|       BARCELONA|04:30 - 05:00|               7|               3|          71801|BARCELONA-

## Fase 2.

Agrupar cada DataFrame por separado para contar el número de viajeros subidos.

In [14]:
df_null = spark.sql("""
  SELECT
    nucleo_cercanias,
    SUM(viajeros_subidos) AS sin_ficha
  FROM joined_data
  WHERE fichas IS NULL
  GROUP BY nucleo_cercanias
  HAVING sin_ficha IS NOT NULL
""")
df_null.createOrReplaceTempView("df_null")

df_not_null = spark.sql("""
  SELECT
    nucleo_cercanias,
    SUM(viajeros_subidos) AS con_ficha
  FROM joined_data
  WHERE fichas IS NOT NULL
  GROUP BY nucleo_cercanias
  HAVING con_ficha IS NOT NULL
""")
df_not_null.createOrReplaceTempView("df_not_null")

print('df_null.show()')
df_null.show()

print('df_not_null.show()')
df_not_null.show()

df_null.show()
+----------------+---------+
|nucleo_cercanias|sin_ficha|
+----------------+---------+
|   SAN SEBASTIÁN|     3499|
|          MADRID|    41267|
|       BARCELONA|    69929|
|    CANTABRIA AM|     7282|
|        ASTURIAS|     1450|
|         SEVILLA|     5270|
|          BILBAO|     9855|
|        VALENCIA|    20646|
|     ASTURIAS AM|     5964|
|          MÁLAGA|     3140|
|   PAIS VASCO AM|     2738|
|           CÁDIZ|     2690|
|         LEON AM|      635|
|          MURCIA|     4667|
|       MURCIA AM|     1072|
|        ZARAGOZA|      173|
+----------------+---------+

df_not_null.show()
+----------------+---------+
|nucleo_cercanias|con_ficha|
+----------------+---------+
|   SAN SEBASTIÁN|    14049|
|          MADRID|   866222|
|       BARCELONA|   299548|
|    CANTABRIA AM|     3519|
|        ASTURIAS|    16173|
|         SEVILLA|    23154|
|          BILBAO|    23539|
|        VALENCIA|    26227|
|     ASTURIAS AM|     1444|
|          MÁLAGA|    24264|
|   PAIS

## Fase 3

Cruzar los resultados agregados y calcular el porcentaje de viajeros sin ficha

In [15]:
df = spark.sql("""
  SELECT
    n.*,
    nn.con_ficha
  FROM df_null n
  JOIN df_not_null nn ON n.nucleo_cercanias = nn.nucleo_cercanias
""")
df.createOrReplaceTempView("joined_null_not_null")

df.show()

+----------------+---------+---------+
|nucleo_cercanias|sin_ficha|con_ficha|
+----------------+---------+---------+
|   SAN SEBASTIÁN|     3499|    14049|
|          MADRID|    41267|   866222|
|       BARCELONA|    69929|   299548|
|    CANTABRIA AM|     7282|     3519|
|        ASTURIAS|     1450|    16173|
|         SEVILLA|     5270|    23154|
|          BILBAO|     9855|    23539|
|        VALENCIA|    20646|    26227|
|     ASTURIAS AM|     5964|     1444|
|          MÁLAGA|     3140|    24264|
|   PAIS VASCO AM|     2738|      168|
|           CÁDIZ|     2690|     5066|
|          MURCIA|     4667|     6405|
|        ZARAGOZA|      173|      844|
+----------------+---------+---------+



In [16]:
df = spark.sql("""
  SELECT
    *,
    ROUND(sin_ficha / (sin_ficha + con_ficha), 2) AS porc_sin_ficha
  FROM joined_null_not_null
  ORDER BY porc_sin_ficha DESC
""")

df.show()

+----------------+---------+---------+--------------+
|nucleo_cercanias|sin_ficha|con_ficha|porc_sin_ficha|
+----------------+---------+---------+--------------+
|   PAIS VASCO AM|     2738|      168|          0.94|
|     ASTURIAS AM|     5964|     1444|          0.81|
|    CANTABRIA AM|     7282|     3519|          0.67|
|        VALENCIA|    20646|    26227|          0.44|
|          MURCIA|     4667|     6405|          0.42|
|           CÁDIZ|     2690|     5066|          0.35|
|          BILBAO|     9855|    23539|           0.3|
|   SAN SEBASTIÁN|     3499|    14049|           0.2|
|       BARCELONA|    69929|   299548|          0.19|
|         SEVILLA|     5270|    23154|          0.19|
|        ZARAGOZA|      173|      844|          0.17|
|          MÁLAGA|     3140|    24264|          0.11|
|        ASTURIAS|     1450|    16173|          0.08|
|          MADRID|    41267|   866222|          0.05|
+----------------+---------+---------+--------------+



## Fase 4

Escribir el resultado final particionado por fecha de ejecución y núcleo de cercanías

In [17]:
now = datetime.now().strftime("%Y%m%d%H%M%S")
df.write.partitionBy('nucleo_cercanias').parquet(f'/content/report/created_at={now}/')

In [18]:
df = spark.read.load('/content/report/')
df.show()

+---------+---------+--------------+--------------+----------------+
|sin_ficha|con_ficha|porc_sin_ficha|    created_at|nucleo_cercanias|
+---------+---------+--------------+--------------+----------------+
|     5964|     1444|          0.81|20241020082441|     ASTURIAS AM|
|     2738|      168|          0.94|20241020082441|   PAIS VASCO AM|
|     2690|     5066|          0.35|20241020082441|           CÁDIZ|
|     9855|    23539|           0.3|20241020082441|          BILBAO|
|     4667|     6405|          0.42|20241020082441|          MURCIA|
|    20646|    26227|          0.44|20241020082441|        VALENCIA|
|      173|      844|          0.17|20241020082441|        ZARAGOZA|
|    69929|   299548|          0.19|20241020082441|       BARCELONA|
|     3499|    14049|           0.2|20241020082441|   SAN SEBASTIÁN|
|     3140|    24264|          0.11|20241020082441|          MÁLAGA|
|     1450|    16173|          0.08|20241020082441|        ASTURIAS|
|    41267|   866222|          0.0